# **Deep Learning Course**

## **Loss Functions and Multilayer Perceptrons (MLP)**

---

### **Student Information:**

- **Name:** *Parsa Ghezelbash*
- **Student Number:** *401110437*

---

### **Assignment Overview**

In this notebook, we will explore various loss functions used in neural networks, with a specific focus on their role in training **Multilayer Perceptrons (MLPs)**. By the end of this notebook, you will have a deeper understanding of:
- Types of loss functions
- How loss functions affect the training process
- The relationship between loss functions and model optimization in MLPs

---

### **Table of Contents**

1. Introduction to Loss Functions
2. Types of Loss Functions
3. Multilayer Perceptrons (MLP)
4. Implementing Loss Functions in MLP
5. Conclusion

---



# 1.Introduction to Loss Functions 

In deep learning, **loss functions** play a crucial role in training models by quantifying the difference between the predicted outputs and the actual targets. Selecting the appropriate loss function is essential for the success of your model. In this assay, we will explore various loss functions available in PyTorch, understand their theoretical backgrounds, and provide you with a scaffolded class to experiment with these loss functions.

Before begining, let's train a simle MLP model using the **L1Loss** function. We'll return to this model later to experiment with different loss functions. We'll start by importing the necessary libraries and defining the model architecture.

First things first, let's talk about **L1Loss**.

### 1. L1Loss (`torch.nn.L1Loss`)
- **Description:** Also known as Mean Absolute Error (MAE), L1Loss computes the average absolute difference between the predicted values and the target values.
- **Use Case:** Suitable for regression tasks where robustness to outliers is desired.

Here is the mathematical formulation of L1Loss:
\begin{equation}
\text{L1Loss} = \frac{1}{n} \sum_{i=1}^{n} |y_{\text{pred}_i} - y_{\text{true}_i}|
\end{equation}

Let's implement a simple MLP model using the L1Loss function.

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from torch.optim import Adam
# Don't be courious about Adam, it's just a fancy name for a fancy optimization algorithm

Here, we'll define a class called `SimpleMLP` that inherits from `nn.Module`. This class can have multiple layers, and we'll use the `nn.Sequential` module to define the layers of the model. The model will have the following architecture:

In [3]:
class SimpleMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_hidden_layers=1, last_layer_activation_fn=nn.ReLU):
        super(SimpleMLP, self).__init__()
        # TODO: Define the layers of the MLP
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_hidden_layers = num_hidden_layers
        self.last_layer_activation_fn = last_layer_activation_fn()

        self.W0 = nn.Parameter(torch.randn(input_dim, hidden_dim))
        self.b0 = nn.Parameter(torch.randn(hidden_dim))

        if num_hidden_layers > 0:
            self.Wh = nn.Parameter(torch.randn(num_hidden_layers-1, hidden_dim, hidden_dim))
            self.bh = nn.Parameter(torch.randn(num_hidden_layers-1, hidden_dim))

        self.wo = nn.Parameter(torch.randn(hidden_dim, output_dim))
        self.bo = nn.Parameter(torch.randn(output_dim))

    def forward(self, x):
        out = x @ self.W0 + self.b0
        out = nn.ReLU()(out)

        for i in range(self.num_hidden_layers-1):
            out = out @ self.Wh[i] + self.bh[i]
            out = nn.ReLU()(out)

        out = out @ self.wo + self.bo
        out = self.last_layer_activation_fn(out)
        
        return out

Now, let's define a class called `SimpleMLP_Loss` that has the following architecture:

In [4]:
from tqdm import tqdm

class SimpleMLPTrainer:
    def __init__(self, model, criterion, optimizer, device='cpu'):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.device = device

    def train(self, train_loader, num_epochs, log=False):
        #TODO: Implement the training loop
        #Note: You should also print the training loss at each epoch, use tqdm for progress bar
        #Note: You should return the training loss at each epoch
        losses = []
        for epoch in tqdm(range(num_epochs)):
            self.model.train()
            epoch_losses = []
            batch_losss = 0

            for X_train, y_train in train_loader:
                X_train, y_train = X_train.to(self.device), y_train.to(self.device)

                y_pred = self.model(X_train).squeeze()
                
                if isinstance(self.criterion, (nn.MSELoss, nn.L1Loss, nn.CrossEntropyLoss, nn.KLDivLoss)):
                    y_train = y_train.float()
                    
                loss = self.criterion(y_pred, y_train)

                epoch_losses.append(loss.item())
                
                self.optimizer.zero_grad()

                loss.backward()

                self.optimizer.step()

                batch_losss += loss.item() * y_train.shape[0]

            losses.append(batch_losss / len(train_loader.dataset))
            if log:
                print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {losses[-1]:.4f}")

        return losses

    def evaluate(self, val_loader, log=False):
        #TODO: Implement the evaluation loop
        #Note: You should return the validation loss and accuracy
        losses = []
        accuracies = []

        losses = 0
        correct = 0
        total = 0

        self.model.eval()

        with torch.no_grad():
            for X_val, y_val in val_loader:

                X_val, y_val = X_val.to(self.device), y_val.to(self.device)

                y_pred = self.model(X_val).squeeze()

                if isinstance(self.criterion, (nn.MSELoss, nn.L1Loss, nn.CrossEntropyLoss)):
                    y_val = y_val.float()

                loss = self.criterion(y_pred, y_val)

                y_pred = (y_pred > 0.5).float().squeeze()
                
                losses += loss.item() * y_val.shape[0]

                correct += (y_pred == y_val).sum().item()

        avg_loss = losses / len(val_loader.dataset)
        accuracy = correct / len(val_loader.dataset)

        if log:
            print(f"Validation Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")
        
        return avg_loss, accuracy

Next, lets test our model using the L1Loss function. You'll use <span style="color:red">*Titanic Dataset*</span> to train the model.


In [5]:
# Load dataset
train_url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
data = pd.read_csv(train_url)

# Preprocessing (simple example)
data = data[['Pclass', 'Sex', 'Age', 'Fare', 'Survived']].dropna()
data['Sex'] = data['Sex'].map({'male': 0, 'female': 1})

X = torch.tensor(data[['Pclass', 'Sex', 'Age', 'Fare']].values, dtype=torch.float)
y = torch.tensor(data['Survived'].values, dtype=torch.long)

# # TODO: Convert the data to PyTorch tensors and create a DataLoader
dataset = TensorDataset(X, y)

# # TODO: Split the data into training and validation sets
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=13)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# TODO: Define the model, criterion, and optimizer
num_epochs = 30
input_dim=4
hidden_dim=16
num_hidden_layers=2
output_dim=1

<div style="text-align: center;"> <span style="color:red; font-size: 26px; font-weight: bold;">Let's train!</span> </div>

In [6]:
from torch.nn import L1Loss

device = 'cuda' if torch.cuda.is_available() else 'cpu'

mlp_model = SimpleMLP(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_hidden_layers=num_hidden_layers)
optimizer = Adam(mlp_model.parameters(), lr=1e-4)
criterion = L1Loss()
L1_trainer = SimpleMLPTrainer(mlp_model, criterion, optimizer)

mlp_model.to(device)
L1_trainer.train(train_loader, num_epochs)

L1_val_loss, L1_val_accuracy = L1_trainer.evaluate(test_loader)

100%|██████████| 30/30 [00:00<00:00, 170.67it/s]


---
# 2. Types of Loss Functions

PyTorch offers a variety of built-in loss functions tailored for different types of problems, such as regression, classification, and more. Below, we discuss several commonly used loss functions, their theoretical foundations, and typical use cases.

### 2. MSELoss (`torch.nn.MSELoss`)
- **Description:** Mean Squared Error (MSE) calculates the average of the squares of the differences between predicted and target values.
- **Use Case:** Commonly used in regression problems where larger errors are significantly penalized.

Here is boring math stuff for MSE:
\begin{equation}
\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_{i} - \hat{y}_{i})^{2}
\end{equation}

<span style="color:red; font-size: 18px; font-weight: bold;">Warning:</span> Don't forget to reinitialize the model before experimenting with different loss functions.

In [7]:
from torch.nn import MSELoss

mlp_model = SimpleMLP(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_hidden_layers=num_hidden_layers)
optimizer = Adam(mlp_model.parameters(), lr=1e-4)
criterion = MSELoss()
MSE_trainer = SimpleMLPTrainer(mlp_model, criterion, optimizer)

mlp_model.to(device)
MSE_trainer.train(train_loader, num_epochs)

MSE_val_loss, MSE_val_accuracy = MSE_trainer.evaluate(test_loader)

100%|██████████| 30/30 [00:00<00:00, 208.05it/s]


### 3. NLLLoss (`torch.nn.NLLLoss`)
- **Description:** Negative Log-Likelihood Loss measures the likelihood of the target class under the predicted probability distribution.
- **Use Case:** Typically used in multi-class classification tasks, especially when combined with `log_softmax` activation.

Here is the mathematical formulation of NLLLoss:
\begin{equation}
\text{NLLLoss} = -\frac{1}{n} \sum_{i=1}^{n} \log(y_{i})
\end{equation}

I hope you note the logarithm in the formula. It's important! 

Why?

In this part, run your training with Relu at last layer. <span style="color:red; font-weight: bold;">Discuss </span> and explain the difference between the results of the two models. Find a proper solution to the problem.


In [8]:
# Run with relu activation function
from torch.nn import NLLLoss

mlp_model = SimpleMLP(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_hidden_layers=num_hidden_layers)
optimizer = Adam(mlp_model.parameters(), lr=1e-4)
criterion = NLLLoss()
NLL_trainer = SimpleMLPTrainer(mlp_model, criterion, optimizer)

mlp_model.to(device)
NLL_trainer.train(train_loader, num_epochs)

# TODO: Evaluate the model
NLL_val_loss, NLL_val_accuracy = NLL_trainer.evaluate(test_loader)

100%|██████████| 30/30 [00:00<00:00, 213.63it/s]


In [9]:
# Run with --- activation function
from torch.nn import NLLLoss

mlp_model = SimpleMLP(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_hidden_layers=num_hidden_layers, last_layer_activation_fn=nn.Sigmoid)
optimizer = Adam(mlp_model.parameters(), lr=1e-4)
criterion = NLLLoss()
NLL_trainer = SimpleMLPTrainer(mlp_model, criterion, optimizer)

mlp_model.to(device)
NLL_trainer.train(train_loader, num_epochs)

# TODO: Evaluate the model
NLL_val_loss, NLL_val_accuracy = NLL_trainer.evaluate(test_loader)

100%|██████████| 30/30 [00:00<00:00, 208.31it/s]


Your reason for your choice:

<div>
**Your answer here**
</div>


### 4. CrossEntropyLoss (`torch.nn.CrossEntropyLoss`)
- **Description:** Combines `LogSoftmax` and `NLLLoss` in one single class. It computes the cross-entropy loss between the target and the output logits.
- **Use Case:** Widely used for multi-class classification problems.

The mathematical formulation of CrossEntropyLoss is as follows:
\begin{equation}
  \text{CrossEntropy}(y, \hat{y}) = - \sum_{i=1}^{C} y_i \log\left(\frac{e^{\hat{y}_i}}{\sum_{j=1}^{C} e^{\hat{y}_j}}\right)
\end{equation}
  where:
  - \( C \) is the number of classes,
  - \( y_i \) is a one-hot encoded target vector (or a scalar class label),
  - \( \hat{y}_i \) represents the logits (unnormalized model outputs) for each class.
  
  In practice, `torch.nn.CrossEntropyLoss` expects raw logits as input and internally applies the softmax function to convert the logits into probabilities, followed by the negative log-likelihood computation.

- **Background:** Cross-entropy measures the difference between the true distribution \( y \) and the predicted distribution \( \hat{y} \). The function minimizes the negative log-probability assigned to the correct class, effectively penalizing predictions that deviate from the true class, making it a standard choice for classification tasks in deep learning.

Now, let's implement a class called `SimpleMLP_Loss` that has the following architecture:


In [10]:
from torch.nn import CrossEntropyLoss

mlp_model = SimpleMLP(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_hidden_layers=num_hidden_layers)
optimizer = Adam(mlp_model.parameters(), lr=1e-4)
criterion = CrossEntropyLoss()
CE_trainer = SimpleMLPTrainer(mlp_model, criterion, optimizer)

# mlp_model.to(device)
CE_trainer.train(train_loader, num_epochs)

# TODO: Evaluate the model
CE_val_loss, CE_val_accuracy = CE_trainer.evaluate(test_loader)

100%|██████████| 30/30 [00:00<00:00, 200.33it/s]



### 5. KLDivLoss (`torch.nn.KLDivLoss`)
- **Description:** Kullback-Leibler Divergence Loss measures how one probability distribution diverges from a second, reference distribution. Unlike other loss functions that focus on classification, KL divergence specifically compares the relative entropy between two distributions. It quantifies the information loss when using the predicted distribution to approximate the true distribution. 

- **Mathematical Function:**
\begin{equation}
  \text{KL}(P \parallel Q) = \sum_{i=1}^{C} P(i) \left( \log P(i) - \log Q(i) \right)
\end{equation}
  where:
  - \( P \) is the target (true) probability distribution,
  - \( Q \) is the predicted distribution (often the output of `log_softmax`),
  - \( C \) is the number of classes.

  KL divergence is always non-negative, and it equals zero if the two distributions are identical. The loss function expects the model's output to be in the form of log-probabilities (using `log_softmax`) and compares this against a target probability distribution, which is typically a normalized distribution (using softmax).

- **Use Case:** KLDivLoss is frequently used in:
  - **Variational Autoencoders (VAEs):** In VAEs, KL divergence is used to measure how much the learned latent space distribution deviates from a prior distribution (often Gaussian).
  - **Knowledge Distillation:** In teacher-student models, KL divergence is used to transfer the "soft" knowledge from a teacher model to a student model by comparing their output probability distributions.
  - **Reinforcement Learning:** It can be used to update policies while minimizing the divergence from a previous policy.

- **Background:** Kullback-Leibler divergence, a core concept in information theory, measures the inefficiency of assuming the predicted distribution \( Q \) when the true distribution is \( P \). It is asymmetric, meaning that \( KL(P \parallel Q) \neq KL(Q \parallel P) \), so the direction of the comparison matters.

Again, in this part, run your training with Relu at last layer. <span style="color:red; font-weight: bold;">Discuss </span> and explain the difference between the results of the two models. Find a proper solution to the problem.


In [11]:
# # Run with relu activation function
# from torch.nn import KLDivLoss

# mlp_model = SimpleMLP(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_hidden_layers=num_hidden_layers, last_layer_activation_fn=nn.Sigmoid)
# optimizer = Adam(mlp_model.parameters(), lr=1e-4)
# criterion = KLDivLoss()
# KL_trainer = SimpleMLPTrainer(mlp_model, criterion, optimizer)

# # mlp_model.to(device)
# KL_trainer.train(train_loader, num_epochs)

# # TODO: Evaluate the model
# KL_val_loss, KL_val_accuracy = KL_trainer.evaluate(test_loader)

In [12]:
# # Run with --- activation function
# from torch.nn import KLDivLoss

# num_epochs = 30
# input_dim=4
# hidden_dim=16
# num_hidden_layers=2
# output_dim=1


# mlp_model = SimpleMLP(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_hidden_layers=num_hidden_layers, last_layer_activation_fn=nn.ReLU)
# optimizer = Adam(mlp_model.parameters(), lr=1e-4)
# criterion = KLDivLoss()
# KL_trainer = SimpleMLPTrainer(mlp_model, criterion, optimizer)

# # mlp_model.to(device)
# KL_trainer.train(train_loader, num_epochs)

# # TODO: Evaluate the model
# KL_val_loss, KL_val_accuracy = KL_trainer.evaluate(test_loader)

Your reason for your choice:

<div>
**Your answer here**
</div>

### 6. CosineEmbeddingLoss (`torch.nn.CosineEmbeddingLoss`)
- **Description:** Measures the cosine similarity between two input tensors, `x1` and `x2`, and computes the loss based on a label `y` that indicates whether the tensors should be similar (`y = 1`) or dissimilar (`y = -1`). Cosine similarity focuses on the angle between vectors, disregarding their magnitude.

- **Mathematical Function:** 
\begin{equation}
  \text{CosineEmbeddingLoss}(x1, x2, y) = 
  \begin{cases} 
  1 - \cos(x_1, x_2), & \text{if } y = 1 \\
  \max(0, \cos(x_1, x_2) - \text{margin}), & \text{if } y = -1
  \end{cases}
\end{equation}
  where $ \cos(x_1, x_2) $ is the cosine similarity between the two vectors, and `margin` is a threshold that determines how dissimilar the vectors should be.

- **Use Case:** Commonly used in tasks like face verification, image similarity, and other scenarios where the relative orientation of vectors (angle) is more important than their length, such as in embeddings and metric learning.

- **Background:** Cosine similarity compares the directional alignment of vectors, making it ideal for high-dimensional data where the magnitude may not be as informative. This loss is particularly useful when training models to learn meaningful embeddings that capture semantic similarity.

You'll become more fimiliar with this loss function in future.

---

In [13]:
# # Run with --- activation function
# from torch.nn import CosineEmbeddingLoss

# num_epochs = 30
# input_dim=4
# hidden_dim=16
# num_hidden_layers=2
# output_dim=1


# mlp_model = SimpleMLP(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_hidden_layers=num_hidden_layers, last_layer_activation_fn=nn.Sigmoid)
# optimizer = Adam(mlp_model.parameters(), lr=1e-4)
# criterion = CosineEmbeddingLoss()
# CoEm_trainer = SimpleMLPTrainer(mlp_model, criterion, optimizer)

# # mlp_model.to(device)
# CoEm_trainer.train(train_loader, num_epochs)

# # TODO: Evaluate the model
# CoEm_val_loss, CoEm_val_accuracy = CoEm_trainer.evaluate(test_loader)

# Regularization in Machine Learning

## Introduction

Regularization is a fundamental technique in machine learning that helps prevent overfitting by adding a penalty to the loss function. This penalty discourages the model from becoming too complex, ensuring better generalization to unseen data. In this notebook, you will explore the concepts of regularization, understand different types of regularization techniques, and apply them using Python's popular libraries.

## What is Regularization?

Regularization involves adding a regularization term to the loss function used to train machine learning models. This term imposes a constraint on the model's coefficients, effectively reducing their magnitude. By doing so, regularization helps in:

- **Preventing Overfitting:** Ensures the model does not become too tailored to the training data.
- **Improving Generalization:** Enhances the model's performance on new, unseen data.
- **Feature Selection:** Especially in L1 regularization, it can drive some coefficients to zero, effectively selecting important features.

## Types of Regularization

There are several types of regularization techniques, each imposing different constraints on the model's parameters:

### 1. L1 Regularization (Lasso)

L1 regularization adds the absolute value of the magnitude of coefficients as a penalty term to the loss function. It can lead to sparse models where some feature coefficients are exactly zero.

### 2. L2 Regularization (Ridge)

L2 regularization adds the squared magnitude of coefficients as a penalty term to the loss function. It tends to shrink the coefficients evenly but does not set them to zero.

### 3. Elastic Net

Elastic Net combines both L1 and L2 regularization penalties. It balances the benefits of both Lasso and Ridge methods, allowing for feature selection and coefficient shrinkage.

## Homework Time!
Import Iris dataset from sklearn.datasets and apply ridge regression with different alpha values. Then, create a gif that shows the changes of the classification boundary with respect to alpha values.

Import the libs that you need and start coding!

In [14]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from PIL import Image
from io import BytesIO
import imageio
import warnings


# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

Load the Iris dataset and select Setosa and Versicolor classes

In [15]:
# 1. Load and Prepare the Iris Dataset
data = load_iris()
df = pd.DataFrame(data=data.data, columns=data.feature_names)
df['target'] = data['target']
# Select only two classes for binary classification (Setosa and Versicolor)
df = df[df['target'].isin([0, 1])]
# Select two features for 2D visualization (Sepal Length and Petal Length)
features = ['sepal length (cm)', 'petal length (cm)']
# Split into training and testing sets
X = df[features]
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Define Function to Plot Decision Boundary

In [16]:
def plot_decision_boundary(model, X, y, alpha):
    # Define the grid (use meshgrid)
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1),
                            np.arange(y_min, y_max, 0.1))

    # Predict over the grid
    grid = np.c_[xx.ravel(), yy.ravel()]

    grid_tensor = torch.tensor(grid, dtype=torch.float32)

    outputs = model(grid_tensor)
    predictions = (outputs > 0.5).float().numpy()

    Z = predictions.reshape(xx.shape)

    # Create a figure
    fig, ax = plt.subplots(figsize=(6, 5))

    # Plot the decision boundary
    ax.contourf(xx, yy, Z, alpha=0.3, levels=[-0.1, 0.1, 1.1], colors=['blue', 'red'])

    # Scatter plot of the training data
    scatter = ax.scatter(
        X[:, 0], X[:, 1], c=y, cmap='bwr', edgecolor='k', s=50
    )

    # Title and labels
    ax.set_title(f'MLP Decision Boundary (alpha={alpha})')
    ax.set_xlabel('Sepal Length (standardized)')
    ax.set_ylabel('Petal Length (standardized)')

    # Remove axes for clarity
    ax.set_xticks([])
    ax.set_yticks([])

    # Tight layout
    plt.tight_layout()

    # Save the plot to a BytesIO object
    buf = BytesIO()
    plt.savefig(buf, format='png')
    plt.close(fig)
    buf.seek(0)
    return Image.open(buf)

Train MLP with Varying Alpha Values and Collect Images

In [17]:
def create_decision_boundary_gif(alpha_values, X_train, y_train, n_neurons):

    # List to store images
    images = []

    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.float32)

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    for idx, alpha in enumerate(alpha_values):
        print(f"Processing alpha={alpha:.4f} ({idx + 1}/{len(alpha_values)})")

        # Create and train the MLP
        mlp = SimpleMLP(X_train.shape[1], hidden_dim=1, output_dim=1, num_hidden_layers=0, last_layer_activation_fn=nn.Sigmoid)
        optimizer = Adam(mlp.parameters(), lr=alpha, weight_decay=1e-4)
        criterion = nn.L1Loss()
        mlp_trainer = SimpleMLPTrainer(mlp, criterion=criterion, optimizer=optimizer)
        
        mlp_trainer.train(train_loader, num_epochs=100)

        # Plot decision boundary and get the image
        img = plot_decision_boundary(mlp, X_train, y_train, alpha)
        images.append(img)

    # Save the images as a GIF
    gif_filename = 'mlp_classification_boundaries.gif'
    images[0].save(
        gif_filename,
        save_all=True,
        append_images=images[1:],
        duration=500,
        loop=0
    )

    print(f"GIF saved as '{gif_filename}'")

    # return the gif
    return gif_filename

## RUN

In [18]:

# Use np.logspace to generate alpha values, with at least 20 values
alpha_values = np.logspace(-2, 3, num=20)
# alpha_values = [0.001]
# Define the number of neurons in the hidden layer
n_neurons = 32
alpha_values
# Create the decision boundary GIF
gif_dir = create_decision_boundary_gif(alpha_values, X_train, y_train, n_neurons)

Processing alpha=0.0100 (1/20)


100%|██████████| 100/100 [00:00<00:00, 1581.53it/s]


Processing alpha=0.0183 (2/20)


100%|██████████| 100/100 [00:00<00:00, 805.06it/s]


Processing alpha=0.0336 (3/20)


...

Processing alpha=545.5595 (19/20)


100%|██████████| 100/100 [00:00<00:00, 1508.25it/s]


Processing alpha=1000.0000 (20/20)


100%|██████████| 100/100 [00:00<00:00, 1501.10it/s]


GIF saved as 'mlp_classification_boundaries.gif'


Your gif should look like this:

<div style="text-align: center;">

### **Multilayer Perceptron Classification Boundaries**

![Classification Boundaries](mlp_classification_boundaries_example.gif)

*Figure 1: Demonstration of classification boundaries created by a Multilayer Perceptron (MLP) model.*

</div>

